In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

In [2]:
import cv2
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import *
from tensorflow.keras.callbacks import *

In [3]:
CLASS_NAMES= ['healthy', 'mild', 'moderate', 'severe']
IMG_HEIGHT = 128
IMG_WIDTH = 128
CHANNELS = 3
NUM_CLASSES = len(CLASS_NAMES)

# **ResNET**

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
#load data
data_x = np.load('data_x_aumentated_normalized.npy') #images
data_y = np.load('data_y_aumentated_normalized.npy') #labels
TOTAL_IMAGES = len(data_y)
print("TOTAL_IMAGES = ", TOTAL_IMAGES)

TOTAL_IMAGES =  2368


In [6]:
data_x[0]

array([[[0.56470588, 0.51764706, 0.65490196],
        [0.63137255, 0.56470588, 0.69411765],
        [0.65490196, 0.59607843, 0.70980392],
        ...,
        [0.23529412, 0.18823529, 0.41568627],
        [0.23529412, 0.18823529, 0.43529412],
        [0.29411765, 0.25098039, 0.47058824]],

       [[0.54117647, 0.46666667, 0.64313725],
        [0.57647059, 0.49803922, 0.65098039],
        [0.62352941, 0.54901961, 0.68235294],
        ...,
        [0.21176471, 0.18431373, 0.41568627],
        [0.21176471, 0.17254902, 0.41960784],
        [0.25490196, 0.21176471, 0.45882353]],

       [[0.47058824, 0.39215686, 0.56862745],
        [0.47843137, 0.39215686, 0.57647059],
        [0.54509804, 0.4627451 , 0.61568627],
        ...,
        [0.29411765, 0.24313725, 0.50196078],
        [0.29803922, 0.27058824, 0.51372549],
        [0.33333333, 0.27843137, 0.5254902 ]],

       ...,

       [[0.55686275, 0.4745098 , 0.63137255],
        [0.57647059, 0.4745098 , 0.63921569],
        [0.58823529, 0

# **Splitting data in train - validation - test**

In [7]:
#percentage of the dataset which will be on TRAIN - TEST - VALIDATION
TRAIN_PERCENTAGE = 0.8
TEST_PERCENTAGE = 0.1
VALIDATION_PERCENTAGE = 0.1

In [8]:
train_size = int(TRAIN_PERCENTAGE * TOTAL_IMAGES)
test_size = int(TEST_PERCENTAGE * TOTAL_IMAGES)
validation_size = int(VALIDATION_PERCENTAGE * TOTAL_IMAGES)

In [9]:
train_x = data_x[:train_size] #taking train images from original data
train_y = data_y[:train_size]
data_x = data_x[train_size:] #removing train images from original data
data_y = data_y[train_size:]

test_x = data_x[:test_size] #taking test images from original data
test_y = data_y[:test_size]
data_x = data_x[test_size:] #removing test images from original data
data_y = data_y[test_size:]

validation_x = data_x #all the remainder images we can put on validation
validation_y = data_y

del data_x #free some unused memory
del data_y

In [10]:
print("train_size = ", len(train_y))
print("test_size = ", len(test_y))
print("validation_size = ", len(validation_y))

train_size =  1894
test_size =  236
validation_size =  238


In [11]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1
from tensorflow.keras.callbacks import EarlyStopping

In [12]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from IPython.display import SVG
import scipy.misc

In [13]:
# from tensorflow.keras.applications import EfficientNetB0
# from tensorflow.keras.applications import EfficientNetB1
# from tensorflow.keras.applications import EfficientNetB2
# from tensorflow.keras.applications import EfficientNetB3
# from tensorflow.keras.applications import EfficientNetB4
# from tensorflow.keras.applications import EfficientNetB5
# from tensorflow.keras.applications import EfficientNetB6
# from tensorflow.keras.applications import EfficientNetB7
# from tensorflow.keras.applications import *

In [14]:
import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

C:\Users\tmcle\.conda\envs\tf\lib\site-packages\keras\backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


In [15]:
model = tf.keras.applications.EfficientNetB1(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(IMG_HEIGHT,IMG_WIDTH, CHANNELS),
    pooling=None,
    classes=NUM_CLASSES,
   # classifier_activation="softmax",
#    **kwargs
)

In [16]:
from tensorflow.keras import models
from tensorflow.keras import layers

new_model = models.Sequential()
new_model.add(model)
#3rd conv layer
new_model.add(tf.keras.layers.Conv2D(32, 3, padding="same"))
new_model.add(tf.keras.layers.BatchNormalization())
new_model.add(tf.keras.layers.Activation("relu"))
new_model.add(tf.keras.layers.MaxPooling2D(2))
new_model.add(tf.keras.layers.Dropout(0.2))
#4th conv layer
new_model.add(tf.keras.layers.Conv2D(64, 3, padding="same"))
new_model.add(tf.keras.layers.BatchNormalization())
new_model.add(tf.keras.layers.Activation("relu"))
new_model.add(tf.keras.layers.MaxPooling2D(2))
new_model.add(tf.keras.layers.Dropout(0.2))
#Flatten Layer
new_model.add(tf.keras.layers.Flatten())

new_model.add(tf.keras.layers.Dense(128, kernel_regularizer =tf.keras.regularizers.l2( l=0.01)))
new_model.add(tf.keras.layers.BatchNormalization())
new_model.add(tf.keras.layers.Activation("relu"))
new_model.add(tf.keras.layers.Dropout(0.2))

#output Dense Layer
new_model.add(tf.keras.layers.Dense(NUM_CLASSES))
new_model.add(tf.keras.layers.Activation('softmax'))
adam = tf.keras.optimizers.Adam(lr=0.001)


C:\Users\tmcle\.conda\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [17]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb1 (Functional)  (None, 4, 4, 1280)       6575239   
                                                                 
 conv2d (Conv2D)             (None, 4, 4, 32)          368672    
                                                                 
 batch_normalization (BatchN  (None, 4, 4, 32)         128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 4, 4, 32)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 32)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 2, 2, 32)          0

In [18]:
epochs = 100

#compila e treina o modelo com os dados carregados
new_model.compile(loss='sparse_categorical_crossentropy', metrics=['acc'], optimizer=adam)
model_trained = new_model.fit(train_x, train_y,epochs=epochs,verbose=2,validation_data=(validation_x, validation_y),shuffle=False)

Epoch 1/100
60/60 - 46s - loss: 1.9855 - acc: 0.4588 - val_loss: 2.1003 - val_acc: 0.2437 - 46s/epoch - 769ms/step
Epoch 2/100
60/60 - 25s - loss: 1.2950 - acc: 0.7339 - val_loss: 2.1405 - val_acc: 0.2437 - 25s/epoch - 418ms/step
Epoch 3/100
60/60 - 25s - loss: 0.9278 - acc: 0.8564 - val_loss: 2.7525 - val_acc: 0.2437 - 25s/epoch - 418ms/step
Epoch 4/100
60/60 - 25s - loss: 0.7037 - acc: 0.9013 - val_loss: 3.7383 - val_acc: 0.2437 - 25s/epoch - 417ms/step
Epoch 5/100
60/60 - 25s - loss: 0.5314 - acc: 0.9382 - val_loss: 3.7200 - val_acc: 0.2437 - 25s/epoch - 416ms/step
Epoch 6/100
60/60 - 25s - loss: 0.4226 - acc: 0.9430 - val_loss: 2.9973 - val_acc: 0.2437 - 25s/epoch - 416ms/step
Epoch 7/100
60/60 - 25s - loss: 0.3668 - acc: 0.9467 - val_loss: 2.7984 - val_acc: 0.2899 - 25s/epoch - 416ms/step
Epoch 8/100
60/60 - 25s - loss: 0.3076 - acc: 0.9535 - val_loss: 61.6397 - val_acc: 0.2143 - 25s/epoch - 416ms/step
Epoch 9/100
60/60 - 25s - loss: 0.2624 - acc: 0.9556 - val_loss: 148.2854 - val

Epoch 72/100
60/60 - 25s - loss: 0.0158 - acc: 0.9984 - val_loss: 5.6931 - val_acc: 0.2437 - 25s/epoch - 415ms/step
Epoch 73/100
60/60 - 25s - loss: 0.0187 - acc: 0.9974 - val_loss: 2.8763 - val_acc: 0.2311 - 25s/epoch - 415ms/step
Epoch 74/100
60/60 - 25s - loss: 0.0229 - acc: 0.9958 - val_loss: 4.3007 - val_acc: 0.2143 - 25s/epoch - 414ms/step
Epoch 75/100
60/60 - 25s - loss: 0.0141 - acc: 0.9989 - val_loss: 3.8779 - val_acc: 0.2941 - 25s/epoch - 415ms/step
Epoch 76/100
60/60 - 25s - loss: 0.0336 - acc: 0.9937 - val_loss: 5.5030 - val_acc: 0.2563 - 25s/epoch - 414ms/step
Epoch 77/100
60/60 - 25s - loss: 0.0349 - acc: 0.9916 - val_loss: 1.6750 - val_acc: 0.2563 - 25s/epoch - 414ms/step
Epoch 78/100
60/60 - 25s - loss: 0.0404 - acc: 0.9921 - val_loss: 4.0556 - val_acc: 0.2899 - 25s/epoch - 414ms/step
Epoch 79/100
60/60 - 25s - loss: 0.0767 - acc: 0.9857 - val_loss: 3.7070 - val_acc: 0.3025 - 25s/epoch - 414ms/step
Epoch 80/100
60/60 - 25s - loss: 0.0270 - acc: 0.9968 - val_loss: 3.6198

In [19]:
model.summary()

Model: "efficientnetb1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 128, 128, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 128, 128, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 128, 128, 3)  0           ['normalization[0][0

 ivation)                                                                                         
                                                                                                  
 block2a_dwconv_pad (ZeroPaddin  (None, 65, 65, 96)  0           ['block2a_expand_activation[0][0]
 g2D)                                                            ']                               
                                                                                                  
 block2a_dwconv (DepthwiseConv2  (None, 32, 32, 96)  864         ['block2a_dwconv_pad[0][0]']     
 D)                                                                                               
                                                                                                  
 block2a_bn (BatchNormalization  (None, 32, 32, 96)  384         ['block2a_dwconv[0][0]']         
 )                                                                                                
          

                                                                                                  
 block2c_activation (Activation  (None, 32, 32, 144)  0          ['block2c_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block2c_se_squeeze (GlobalAver  (None, 144)         0           ['block2c_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block2c_se_reshape (Reshape)   (None, 1, 1, 144)    0           ['block2c_se_squeeze[0][0]']     
                                                                                                  
 block2c_se_reduce (Conv2D)     (None, 1, 1, 6)      870         ['block2c_se_reshape[0][0]']     
          

 block3b_se_reshape (Reshape)   (None, 1, 1, 240)    0           ['block3b_se_squeeze[0][0]']     
                                                                                                  
 block3b_se_reduce (Conv2D)     (None, 1, 1, 10)     2410        ['block3b_se_reshape[0][0]']     
                                                                                                  
 block3b_se_expand (Conv2D)     (None, 1, 1, 240)    2640        ['block3b_se_reduce[0][0]']      
                                                                                                  
 block3b_se_excite (Multiply)   (None, 16, 16, 240)  0           ['block3b_activation[0][0]',     
                                                                  'block3b_se_expand[0][0]']      
                                                                                                  
 block3b_project_conv (Conv2D)  (None, 16, 16, 40)   9600        ['block3b_se_excite[0][0]']      
          

 block4a_se_reduce (Conv2D)     (None, 1, 1, 10)     2410        ['block4a_se_reshape[0][0]']     
                                                                                                  
 block4a_se_expand (Conv2D)     (None, 1, 1, 240)    2640        ['block4a_se_reduce[0][0]']      
                                                                                                  
 block4a_se_excite (Multiply)   (None, 8, 8, 240)    0           ['block4a_activation[0][0]',     
                                                                  'block4a_se_expand[0][0]']      
                                                                                                  
 block4a_project_conv (Conv2D)  (None, 8, 8, 80)     19200       ['block4a_se_excite[0][0]']      
                                                                                                  
 block4a_project_bn (BatchNorma  (None, 8, 8, 80)    320         ['block4a_project_conv[0][0]']   
 lization)

...

 ization)                                                                                         
                                                                                                  
 block5b_expand_activation (Act  (None, 8, 8, 672)   0           ['block5b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block5b_dwconv (DepthwiseConv2  (None, 8, 8, 672)   16800       ['block5b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block5b_bn (BatchNormalization  (None, 8, 8, 672)   2688        ['block5b_dwconv[0][0]']         
 )                                                                                                
          

 block5d_dwconv (DepthwiseConv2  (None, 8, 8, 672)   16800       ['block5d_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block5d_bn (BatchNormalization  (None, 8, 8, 672)   2688        ['block5d_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block5d_activation (Activation  (None, 8, 8, 672)   0           ['block5d_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block5d_se_squeeze (GlobalAver  (None, 672)         0           ['block5d_activation[0][0]']     
 agePoolin

 )                                                                                                
                                                                                                  
 block6b_se_squeeze (GlobalAver  (None, 1152)        0           ['block6b_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block6b_se_reshape (Reshape)   (None, 1, 1, 1152)   0           ['block6b_se_squeeze[0][0]']     
                                                                                                  
 block6b_se_reduce (Conv2D)     (None, 1, 1, 48)     55344       ['block6b_se_reshape[0][0]']     
                                                                                                  
 block6b_se_expand (Conv2D)     (None, 1, 1, 1152)   56448       ['block6b_se_reduce[0][0]']      
          

 block6d_se_reshape (Reshape)   (None, 1, 1, 1152)   0           ['block6d_se_squeeze[0][0]']     
                                                                                                  
 block6d_se_reduce (Conv2D)     (None, 1, 1, 48)     55344       ['block6d_se_reshape[0][0]']     
                                                                                                  
 block6d_se_expand (Conv2D)     (None, 1, 1, 1152)   56448       ['block6d_se_reduce[0][0]']      
                                                                                                  
 block6d_se_excite (Multiply)   (None, 4, 4, 1152)   0           ['block6d_activation[0][0]',     
                                                                  'block6d_se_expand[0][0]']      
                                                                                                  
 block6d_project_conv (Conv2D)  (None, 4, 4, 192)    221184      ['block6d_se_excite[0][0]']      
          

                                                                                                  
 block7a_se_excite (Multiply)   (None, 4, 4, 1152)   0           ['block7a_activation[0][0]',     
                                                                  'block7a_se_expand[0][0]']      
                                                                                                  
 block7a_project_conv (Conv2D)  (None, 4, 4, 320)    368640      ['block7a_se_excite[0][0]']      
                                                                                                  
 block7a_project_bn (BatchNorma  (None, 4, 4, 320)   1280        ['block7a_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block7b_expand_conv (Conv2D)   (None, 4, 4, 1920)   614400      ['block7a_project_bn[0][0]']     
          

In [20]:
def getLayerIndexByName(model, layername):
    for idx, layer in enumerate(model.layers):
        if layer.name == layername:
            return idx

In [21]:
print(getLayerIndexByName(new_model, 'flatten')) #just used to get the index of flatten layer

11


In [22]:
model_name = f'efficientNetB1-baseline-holdout-{epochs}epochs_dropout_batch_augmented.npy'
new_model.save(model_name)

INFO:tensorflow:Assets written to: efficientNetB1-baseline-holdout-100epochs_dropout_batch_augmented.npy\assets


INFO:tensorflow:Assets written to: efficientNetB1-baseline-holdout-100epochs_dropout_batch_augmented.npy\assets


TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

# **Random Forest - flatting**

In [ ]:
new_model_rf = keras.Model(inputs=new_model.input, outputs=new_model.get_layer(index=11).output)
featureVector = new_model_rf.predict(train_x)

In [ ]:
featureVector2 = new_model_rf.predict(test_x)

In [ ]:
print(featureVector.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf=RandomForestClassifier(n_estimators=100)

clf.fit(featureVector,train_y)



In [ ]:
test_x.shape

In [ ]:
y_pred=clf.predict(featureVector2)

In [ ]:
featureVector.shape

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(test_y, y_pred))

# **Testes**

In [ ]:
import matplotlib.pyplot as plt

accuracy = model_trained.history['acc']
val_accuracy = model_trained.history['val_acc']
loss = model_trained.history['loss']
val_loss = model_trained.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b', label='Acurácia de treino')
plt.plot(epochs, val_accuracy, 'g', label='Acurácia de validação')
plt.title('Acurácia de treino e validação')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Perda no treino')
plt.plot(epochs, val_loss, 'g', label='Perda na validação')
plt.title('Perda na validação e treino')
plt.legend()
plt.show()

In [ ]:
#avalia a fase de teste

model_loss, model_accuracy = new_model.evaluate(test_x, test_y, verbose=1)

#mostra o resultado
print('Test loss:', model_loss)
print('Test accuracy:', model_accuracy)

In [ ]:
#faz a predição das imagens

from tensorflow.keras.utils import to_categorical

pred = new_model.predict(test_x, verbose=0)

pred_probs = np.argmax(pred,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
cm = confusion_matrix(test_y, pred_probs)

In [ ]:
#gera os valores de falso positivo, falso negativo, verdadeiro positivo e verdadeiro negativo
fp = cm.sum(axis=0) - np.diag(cm)  
fn = cm.sum(axis=1) - np.diag(cm)
tp = np.diag(cm)
tn = cm.sum() - (fp + fn + tp)

f1score = f1_score(test_y, pred_probs, average='weighted')
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
accuracy    = (tp + tn) / (tp + tn + fp + fn)
precision   = tp / (tp + fp)


print("F1 Score:", f1score)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Precision:  ", precision)
print("Accuracy:   ", accuracy)